# First attempt at using ML to understand what features predict the number of pathogens/parasites/insects on a plant. 

Tasks:
1. import data `"C:\Users\Seth\Dropbox\_LEARNING_\projects\Plant\data\BIfloraexplorer\data"`
2. explore data
3. feature selection
4. create a model to predict plant genome size
5. evaluate model

In [51]:
# install seaborn
# %pip install seaborn
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


In [52]:
# Load the data from the data folder, assess each file to determine which one to use
datadir = './data/BIfloraexplorer/data/'


# read in every file in the data directory into its own dataframe
def read_data(datadir):
    data = []
    encodings = ['utf-8', 'latin-1', 'iso-8859-1', 'cp1252']
    for file in os.listdir(datadir):
        print(f"Attempting to read file: {file}")
        for encoding in encodings:
            try:
                df = pd.read_csv(os.path.join(datadir, file), encoding=encoding)
                data.append(df)
                print(f"Successfully read {file} with encoding: {encoding}")
                break
            except UnicodeDecodeError:
                print(f"Failed to read {file} with encoding: {encoding}")
        else:
            print(f"Failed to read {file} with all attempted encodings")
    return data

data = read_data(datadir)

Attempting to read file: GS_Kew_BI.csv
Failed to read GS_Kew_BI.csv with encoding: utf-8
Successfully read GS_Kew_BI.csv with encoding: latin-1
Attempting to read file: BI_main.csv
Failed to read BI_main.csv with encoding: utf-8
Successfully read BI_main.csv with encoding: latin-1
Attempting to read file: .~lock.BI_main.csv#
Successfully read .~lock.BI_main.csv# with encoding: utf-8
Attempting to read file: chrom_num_BI.csv
Failed to read chrom_num_BI.csv with encoding: utf-8
Successfully read chrom_num_BI.csv with encoding: latin-1
Attempting to read file: GS_BI.csv
Failed to read GS_BI.csv with encoding: utf-8
Successfully read GS_BI.csv with encoding: latin-1


Errors in import bc encoding is all in latin-1. Need to change to utf-8.


In [53]:
def read_data(datadir):
    data = []
    for file in os.listdir(datadir):
        file_path = os.path.join(datadir, file)
        try:
            df = pd.read_csv(file_path, encoding='latin-1')
            data.append(df)
            print(f"Successfully read {file}")
        except Exception as e:
            print(f"Error reading {file}: {str(e)}")
    return data
data = read_data(datadir)


Successfully read GS_Kew_BI.csv
Successfully read BI_main.csv
Successfully read .~lock.BI_main.csv#
Successfully read chrom_num_BI.csv
Successfully read GS_BI.csv


In [54]:
# examine the number of species, size of the dataframes

for df in data:
    print(df.shape)
    print(df['kew_id'].nunique()) # number of unique species taxon_name is the same

(594, 18)
467
(3227, 83)
3227
(0, 5)


KeyError: 'kew_id'

In [ ]:
# examine the column names
for df in data:
    print(df.columns)

Index(['kew_id', 'taxon_name', 'taxon_name_binom', 'authors', 'MSB_number',
       'standard_species', 'standard_2C_pg', 'buffer', 'buffer_composition',
       'buffer_origin', 'GS_2C_pg', 'GS_1C_pg', 'GS_2C_Mbp', 'GS_1C_Mbp',
       'cv_sample', 'cv_standard', 'matched_by_synonym', 'cited_name'],
      dtype='object')
Index(['kew_id', 'unclear_species_marker', 'extinct_species_marker',
       'taxon_name', 'taxon_name_binom', 'authors', 'taxon_name_WCVP',
       'authors_WCVP', 'order', 'family', 'genus', 'subgenus', 'section',
       'subsection', 'series', 'species', 'group', 'aggregate',
       'members_of_agg.', 'taxonomic_status', 'accepted_kew_id',
       'accepted_name', 'accepted_authors', 'imperfect_match_with_Stace_IV',
       'WCVP_URL', 'POWO_URL', 'IPNI_URL', 'accepted_WCVP_URL',
       'StaceIV_nativity', 'Atlas_nativity_viaALIENATT_PLANTATT',
       'Stace_Crawley_nativity_aliens', 'SLA', 'LDMC', 'seed_mass',
       'leaf_area', 'mean_veg_height', 'max_veg_height', 'L_P

In [ ]:
# are there any plants in GS_Kew_BI.csv that are not in GS_BI.csv or vice versa? This is the first and last df in the list

# check for missing species in GS_Kew_BI.csv
missing_species = data[0][~data[0]['kew_id'].isin(data[-1]['kew_id'])]

In [ ]:
missing_species

,kew_id,taxon_name,taxon_name_binom,authors,MSB_number,standard_species,standard_2C_pg,buffer,buffer_composition,buffer_origin,GS_2C_pg,GS_1C_pg,GS_2C_Mbp,GS_1C_Mbp,cv_sample,cv_standard,matched_by_synonym,cited_name


In [ ]:
# check for missing species in GS_BI.csv.csv
missing_species = data[-1][~data[-1]['kew_id'].isin(data[0]['kew_id'])]
missing_species

,kew_id,taxon_name,taxon_name_binom,authors,GS_2C_pg,GS_1C_pg,GS_2C_Mbp,GS_1C_Mbp,from_BI_material,data_source,ID
594,60468511-2,Abies alba Mill.,Abies alba,Mill.,34.54,17.27,33783.36,16891.68,n,marda et al. 2019,2
595,1044174-2,Abutilon theophrasti Medik.,Abutilon theophrasti,Medik.,1.90,0.95,1861.69,930.84,n,marda et al. 2019,2
597,781412-1,Acer negundo L.,Acer negundo,L.,0.93,0.46,906.36,453.18,n,marda et al. 2019,2
598,781455-1,Acer platanoides L.,Acer platanoides,L.,1.27,0.63,1239.55,619.77,n,marda et al. 2019,2
600,2332-2,Achillea nobilis L.,Achillea nobilis,L.,4.63,2.31,4523.38,2261.69,n,marda et al. 2019,2
...,...,...,...,...,...,...,...,...,...,...,...
4387,260893-1,Xanthium strumarium L.,Xanthium strumarium,L.,6.40,3.20,6272.00,3136.00,n,C-ValueDB,4
4388,325010-2,Yucca gloriosa L.,Yucca gloriosa,L.,6.10,3.05,5978.00,2989.00,n,C-ValueDB,4
4389,603593-1,Zannichellia palustris L.,Zannichellia palustris,L.,4.69,2.34,4592.00,2296.00,n,C-ValueDB,4
4390,89403-1,Zantedeschia aethiopica (L.) Spreng.,Zantedeschia aethiopica,(L.) Spreng.,4.60,2.30,4508.00,2254.00,n,C-ValueDB,4


Lots of data in GS_BI.csv that are not in Kew file. All of the data seems to be present in `BI_main.csv`. 

In [ ]:
df = data[1]

In [ ]:
# get column names and then edit for needed columns
colnames = df.columns
print(colnames)

Index(['kew_id', 'unclear_species_marker', 'extinct_species_marker',
       'taxon_name', 'taxon_name_binom', 'authors', 'taxon_name_WCVP',
       'authors_WCVP', 'order', 'family', 'genus', 'subgenus', 'section',
       'subsection', 'series', 'species', 'group', 'aggregate',
       'members_of_agg.', 'taxonomic_status', 'accepted_kew_id',
       'accepted_name', 'accepted_authors', 'imperfect_match_with_Stace_IV',
       'WCVP_URL', 'POWO_URL', 'IPNI_URL', 'accepted_WCVP_URL',
       'StaceIV_nativity', 'Atlas_nativity_viaALIENATT_PLANTATT',
       'Stace_Crawley_nativity_aliens', 'SLA', 'LDMC', 'seed_mass',
       'leaf_area', 'mean_veg_height', 'max_veg_height', 'L_PLANTATT',
       'F_PLANTATT', 'R_PLANTATT', 'N_PLANTATT', 'S_PLANTATT', 'L_Doring',
       'F_Doring', 'R_Doring', 'N_Doring', 'S_Doring', 'T_Doring', 'ECPE_CSR',
       'predicted_CSR', 'growth_form', 'succulence', 'life_form', 'biome',
       'origin', 'TDWG_level_1_code', 'GB_Man_hectads_post2000',
       'Ire_hecta

In [ ]:
xcols = ['StaceIV_nativity', 'Atlas_nativity_viaALIENATT_PLANTATT',
       'Stace_Crawley_nativity_aliens', 'SLA', 'LDMC', 'seed_mass',
       'leaf_area', 'mean_veg_height', 'max_veg_height', 'L_PLANTATT',
       'F_PLANTATT', 'R_PLANTATT', 'N_PLANTATT', 'S_PLANTATT', 'L_Doring',
       'F_Doring', 'R_Doring', 'N_Doring', 'S_Doring', 'T_Doring', 'ECPE_CSR',
       'predicted_CSR', 'growth_form', 'succulence', 'life_form', 'biome',
       'origin', 'TDWG_level_1_code', 'GB_Man_hectads_post2000',
       'Ire_hectads_post2000', 'CI_hectads_post2000',
       'GB_Man_hectads_1987_1999', 'Ire_hectads_1987_1999',
       'CI_hectads_1987_1999', 'GB_Man_hectads_2000_2009',
       'Ire_hectads_2000_2009', 'CI_hectads_2000_2009',
       'GB_Man_hectads_2010_2019', 'Ire_hectads_2010_2019',
       'CI_hectads_2010_2019', 'hybrid_propensity', 'scaled_hybrid_propensity'
       ]

ycols = ['GS_1C_pg', 'GS_2C_pg',
       'GS_1C_Mbp', 'GS_2C_Mbp', 
       'sporophytic_chromosome_number', 'infraspecific_variation_chrom_number']

idcols = ['kew_id', 'taxon_name']

In [ ]:
X = df[xcols]
y = df[ycols]

In [ ]:
# Encode categorical variables (one-hot encoding)
# function to identify categorical columns
def cat_cols(df):
    return df.select_dtypes(include=['object']).columns.tolist()

catcols = cat_cols(X)
catcols

['StaceIV_nativity',
 'Atlas_nativity_viaALIENATT_PLANTATT',
 'Stace_Crawley_nativity_aliens',
 'ECPE_CSR',
 'predicted_CSR',
 'growth_form',
 'succulence',
 'life_form',
 'biome',
 'origin',
 'TDWG_level_1_code']

In [ ]:
import sklearn
# one-hot encode the categorical columns
def one_hot_encode(df, categorical_columns):
    # Check sklearn version
    sklearn_version = sklearn.__version__
    is_old_version = tuple(map(int, sklearn_version.split('.'))) < (0, 22)

    # Initialize the OneHotEncoder
    if is_old_version:
        print(f"Using old sklearn version: {sklearn_version}")
        encoder = OneHotEncoder(handle_unknown='ignore')
    else:
        print(f"Using new sklearn version: {sklearn_version}")
        encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    
    # Fit and transform the categorical columns
    encoded_cols = encoder.fit_transform(df[categorical_columns])
    
    # Convert to dense array if using an old version
    if is_old_version:
        encoded_cols = encoded_cols.toarray()
    
    # Get the new column names
    if hasattr(encoder, 'get_feature_names_out'):
        new_columns = encoder.get_feature_names_out(categorical_columns)
    elif hasattr(encoder, 'get_feature_names'):
        new_columns = encoder.get_feature_names(categorical_columns)
    else:
        new_columns = [f"{col}_{val}" for col, vals in zip(categorical_columns, encoder.categories_) 
                       for val in vals]
    
    # Create a new dataframe with encoded values
    encoded_df = pd.DataFrame(encoded_cols, columns=new_columns, index=df.index)
    
    # Concatenate the encoded df with the original df, dropping the original categorical columns
    final_df = pd.concat([df.drop(columns=categorical_columns), encoded_df], axis=1)
    
    return final_df

# 
X_encoded = one_hot_encode(X, catcols)


Using new sklearn version: 1.4.2


In [ ]:
def scale_numeric_features(df):
    # Identify numeric columns
    numeric_columns = df.select_dtypes(include=['int64', 'float64']).columns
    
    # Identify non-numeric columns
    non_numeric_columns = df.columns.difference(numeric_columns)
    
    # Initialize the StandardScaler
    scaler = StandardScaler()
    
    # Fit and transform the numeric columns
    scaled_data = scaler.fit_transform(df[numeric_columns])
    
    # Create a new dataframe with scaled values
    scaled_df = pd.DataFrame(scaled_data, columns=numeric_columns, index=df.index)
    
    # Concatenate the scaled df with the non-numeric columns from the original df
    final_df = pd.concat([scaled_df, df[non_numeric_columns]], axis=1)
    
    return final_df

# Assuming X is your feature dataframe
X_scaled = scale_numeric_features(X)

# Print information about the scaling
print("Original X shape:", X.shape)
print("Scaled X shape:", X_scaled.shape)
print("\nFirst few rows of scaled X:")
print(X_scaled.head())

# Optional: Check the mean and standard deviation of scaled numeric features
numeric_columns = X_scaled.select_dtypes(include=['int64', 'float64']).columns
print("\nMean of scaled numeric features:")
print(X_scaled[numeric_columns].mean())
print("\nStandard deviation of scaled numeric features:")
print(X_scaled[numeric_columns].std())


Original X shape: (3227, 42)
Scaled X shape: (3227, 42)

First few rows of scaled X:
        SLA      LDMC  seed_mass  leaf_area  mean_veg_height  max_veg_height  \
0 -1.242879  3.393864   0.009248  -0.303704         7.337411        5.738640   
1  4.615063       NaN  -0.023872        NaN         7.879354        6.364813   
2 -1.329465       NaN   0.017504        NaN         3.898609        3.233949   
3 -1.363572       NaN  -0.073651  -0.322489         2.957833        2.071057   
4 -0.993865       NaN  -0.052019  -0.319492         4.144379        8.601143   

   L_PLANTATT  F_PLANTATT  R_PLANTATT  N_PLANTATT  ...  ECPE_CSR  \
0         NaN         NaN         NaN         NaN  ...       NaN   
1         NaN         NaN         NaN         NaN  ...       NaN   
2         NaN         NaN         NaN         NaN  ...       NaN   
3         NaN         NaN         NaN         NaN  ...       NaN   
4         NaN         NaN         NaN         NaN  ...       NaN   

   StaceIV_nativity  Stac

In [ ]:
# scale
X_scaled = scale_numeric_features(X)
# one-hot encode
X_encoded = one_hot_encode(X_scaled, catcols)

Using new sklearn version: 1.4.2


In [ ]:
print(y.columns)

Index(['GS_1C_pg', 'GS_2C_pg', 'GS_1C_Mbp', 'GS_2C_Mbp',
       'sporophytic_chromosome_number',
       'infraspecific_variation_chrom_number'],
      dtype='object')


In [ ]:
# 'sporophytic_chromosome_number',     'infraspecific_variation_chrom_number' are variable in format. Drop them
y = y.drop(columns=['sporophytic_chromosome_number', 'infraspecific_variation_chrom_number'])

In [ ]:
def scale_target_df(y_df):
    # Initialize StandardScaler
    scaler = StandardScaler()
    
    # Fit and transform the DataFrame
    y_scaled = pd.DataFrame(scaler.fit_transform(y_df), 
                            columns=y_df.columns, 
                            index=y_df.index)
    
    # Return scaled y DataFrame and the scaler for inverse transformation later
    return y_scaled, scaler

# Usage
# Assuming y is your target DataFrame
y_scaled, y_scaler = scale_target_df(y)

print("Original y head:")
print(y.head())
print("\nScaled y head:")
print(y_scaled.head())

# After making predictions, to inverse transform:
# y_pred_original = pd.DataFrame(y_scaler.inverse_transform(y_pred_scaled), 
#                                columns=y_scaled.columns, 
#                                index=y_scaled.index)


Original y head:
   GS_1C_pg  GS_2C_pg  GS_1C_Mbp  GS_2C_Mbp
0     17.27     34.54   16891.68   33783.36
1       NaN       NaN        NaN        NaN
2     18.14     36.27   17738.00   35476.00
3     17.24     34.47   16856.00   33712.00
4     17.55     35.10   17163.90   34327.80

Scaled y head:
   GS_1C_pg  GS_2C_pg  GS_1C_Mbp  GS_2C_Mbp
0  2.321673  2.321720   2.319485   2.319491
1       NaN       NaN        NaN        NaN
2  2.467286  2.466490   2.464187   2.464193
3  2.316652  2.315862   2.313384   2.313390
4  2.368537  2.368582   2.366028   2.366034


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=0)


In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

StandardScaler()

In [ ]:

from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import RFECV
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer

def select_features_multi_y(X, Y, n_estimators=100, cv=5, nan_strategy='drop'):
    if not isinstance(Y, pd.DataFrame):
        Y = pd.DataFrame(Y, columns=['target'])
    
    all_selected_features = set()
    X_selected_dict = {}
    
    for col in Y.columns:
        print(f"\nProcessing target: {col}")
        y = Y[col]
        
        # Handle NaNs in y
        if nan_strategy == 'drop':
            mask = ~y.isna()
            X_temp = X[mask]
            y_temp = y[mask]
        elif nan_strategy == 'impute':
            imputer = SimpleImputer(strategy='mean')
            y_temp = pd.Series(imputer.fit_transform(y.values.reshape(-1, 1)).flatten(), index=y.index)
            X_temp = X
        else:
            raise ValueError("nan_strategy must be either 'drop' or 'impute'")
        
        # Step 1: Initial feature importance using Random Forest
        rf = RandomForestRegressor(n_estimators=n_estimators, random_state=42)
        rf.fit(X_temp, y_temp)
        
        importances = pd.DataFrame({'feature': X_temp.columns, 'importance': rf.feature_importances_})
        importances = importances.sort_values('importance', ascending=False).reset_index(drop=True)
        
        print("Top 10 features by importance:")
        print(importances.head(10))
        
        # Step 2: Recursive Feature Elimination with Cross-validation
        rfecv = RFECV(estimator=RandomForestRegressor(n_estimators=n_estimators, random_state=42), 
                      step=1, cv=cv, scoring='neg_mean_squared_error', n_jobs=-1)
        rfecv.fit(X_temp, y_temp)
        
        selected_features = X_temp.columns[rfecv.support_]
        all_selected_features.update(selected_features)
        
        print(f"Optimal number of features: {rfecv.n_features_}")
        print(f"Selected features: {list(selected_features)}")
        
        # Step 3: Evaluate performance
        X_selected = X_temp[selected_features]
        scores = cross_val_score(RandomForestRegressor(n_estimators=n_estimators, random_state=42), 
                                 X_selected, y_temp, cv=cv, scoring='neg_mean_squared_error')
        mse_scores = -scores
        rmse_scores = np.sqrt(mse_scores)
        
        print(f"Cross-validated RMSE with selected features: {rmse_scores.mean():.4f} (+/- {rmse_scores.std() * 2:.4f})")
        
        X_selected_dict[col] = X_selected
    
    print(f"\nTotal unique features selected across all targets: {len(all_selected_features)}")
    print(f"Combined selected features: {list(all_selected_features)}")
    
    return X[list(all_selected_features)], list(all_selected_features), X_selected_dict




In [ ]:
# Usage
# Assuming X_scaled_encoded is your scaled and encoded feature DataFrame, and Y is your target DataFrame
X_selected, all_selected_features, X_selected_dict = select_features_multi_y(X_encoded, y, nan_strategy='impute')

print("\nShape of original dataset:", X_encoded.shape)
print("Shape of dataset with all selected features:", X_selected.shape)


Processing target: GS_1C_pg
Top 10 features by importance:
                                        feature  importance
0                            life_form_geophyte    0.127469
1             Stace_Crawley_nativity_aliens_Neo    0.101421
2  life_form_epiphyte / parasite / phanerophyte    0.087203
3                                     seed_mass    0.078583
4                      GB_Man_hectads_1987_1999    0.037048
5                                           SLA    0.028466
6                                     leaf_area    0.026143
7                                max_veg_height    0.022842
8                               mean_veg_height    0.020808
9                              growth_form_Fern    0.020398


KeyboardInterrupt: 

### Above is very slow. Impute probably not the right approach. Use 'drop' or ignore that argument to default to 'drop'.

In [55]:
# Usage
# Assuming X_scaled_encoded is your scaled and encoded feature DataFrame, and Y is your target DataFrame

# operate on training data
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, random_state=0)

X_selected_drop, all_selected_features_drop, X_selected_dict_drop = select_features_multi_y(X_encoded, y, nan_strategy='drop')

print("\nShape of original dataset:", X_encoded.shape)
print("Shape of dataset with all selected features:", X_selected_drop.shape)


Processing target: GS_1C_pg
Top 10 features by importance:
                                        feature  importance
0                            life_form_geophyte    0.195440
1             Stace_Crawley_nativity_aliens_Neo    0.121455
2  life_form_epiphyte / parasite / phanerophyte    0.088456
3                                     seed_mass    0.069139
4                                           SLA    0.028863
5                              growth_form_Fern    0.028292
6                               mean_veg_height    0.023060
7                                max_veg_height    0.020567
8                                     leaf_area    0.019513
9                                    L_PLANTATT    0.019172
Optimal number of features: 158
Selected features: ['SLA', 'LDMC', 'seed_mass', 'leaf_area', 'mean_veg_height', 'max_veg_height', 'L_PLANTATT', 'F_PLANTATT', 'R_PLANTATT', 'N_PLANTATT', 'S_PLANTATT', 'L_Doring', 'F_Doring', 'R_Doring', 'N_Doring', 'S_Doring', 'T_Doring', 'GB_Man_

### Above also takes a very long time. 

Shape of original dataset: (3227, 515)
Shape of dataset with all selected features: (3227, 381)


In [58]:
X_selected_dict_drop['GS_1C_pg'].head()

,SLA,LDMC,seed_mass,leaf_area,mean_veg_height,max_veg_height,L_PLANTATT,F_PLANTATT,R_PLANTATT,N_PLANTATT,...,origin_nan,TDWG_level_1_code_1,"TDWG_level_1_code_1, 2","TDWG_level_1_code_1, 2, 3","TDWG_level_1_code_1, 3",TDWG_level_1_code_2,TDWG_level_1_code_3,TDWG_level_1_code_7,TDWG_level_1_code_8,TDWG_level_1_code_nan
0,-1.242879,3.393864,0.009248,-0.303704,7.337411,5.738640,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-1.329465,NaN,0.017504,NaN,3.898609,3.233949,NaN,NaN,NaN,NaN,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.363572,NaN,-0.073651,-0.322489,2.957833,2.071057,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,-0.993865,NaN,-0.052019,-0.319492,4.144379,8.601143,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,NaN,NaN,0.026003,NaN,7.684442,5.917546,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [60]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

def run_models(X, y, X_dict, n_estimators=100, cv=5):
    results = {}
    
    if not isinstance(y, pd.DataFrame):
        y = pd.DataFrame(y, columns=['target'])
    
    for target in y.columns:
        print(f"\nModeling for target: {target}")
        y_target = y[target].dropna()
        X_target = X_dict[target]
        X_all = X.loc[y_target.index]
        
        models = {
            'Random Forest (Selected Features)': RandomForestRegressor(n_estimators=n_estimators, random_state=42),
            'Gradient Boosting (Selected Features)': GradientBoostingRegressor(n_estimators=n_estimators, random_state=42),
            'Random Forest (All Features)': RandomForestRegressor(n_estimators=n_estimators, random_state=42),
            'Gradient Boosting (All Features)': GradientBoostingRegressor(n_estimators=n_estimators, random_state=42)
        }
        
        target_results = {}
        
        for name, model in models.items():
            if 'Selected Features' in name:
                X_use = X_target
            else:
                X_use = X_all
            
            # Cross-validation
            scores = cross_val_score(model, X_use, y_target, cv=cv, scoring='neg_mean_squared_error')
            rmse_scores = np.sqrt(-scores)
            
            # Train on full dataset and get R2 score
            model.fit(X_use, y_target)
            y_pred = model.predict(X_use)
            r2 = r2_score(y_target, y_pred)
            
            target_results[name] = {
                'RMSE': rmse_scores.mean(),
                'RMSE_std': rmse_scores.std() * 2,
                'R2': r2
            }
            
            print(f"{name}:")
            print(f"  RMSE: {rmse_scores.mean():.4f} (+/- {rmse_scores.std() * 2:.4f})")
            print(f"  R2 Score: {r2:.4f}")
        
        results[target] = target_results
    
    return results

# Run the models
model_results = run_models(X_selected_drop, y, X_selected_dict_drop)

# Print summary
print("\nSummary of Results:")
for target, target_results in model_results.items():
    print(f"\nTarget: {target}")
    for model, metrics in target_results.items():
        print(f"  {model}:")
        print(f"    RMSE: {metrics['RMSE']:.4f} (+/- {metrics['RMSE_std']:.4f})")
        print(f"    R2 Score: {metrics['R2']:.4f}")


Modeling for target: GS_1C_pg
Random Forest (Selected Features):
  RMSE: 4.9008 (+/- 1.0418)
  R2 Score: 0.9131


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/ensemble/_gb.py", line 659, in fit
    X, y = self._validate_data(
           ^^^^^^^^^^^^^^^^^^^^
  File "/Library/Fr

Above errors on gradient boosting bc of NAs. Seems like the random forest worked fine. Below using HistGradientBoosting which can handle NAs.

Additional problem. I ran on the whole dataset above, when I should be running based on the training set.

In [62]:
# operate on training data
X_selected_drop, all_selected_features_drop, X_selected_dict_drop = select_features_multi_y(X_train, y_train, nan_strategy='drop')

print("\nShape of original dataset:", X_encoded.shape)
print("Shape of dataset with all selected features:", X_selected_drop.shape)


Processing target: GS_1C_pg
Top 10 features by importance:
                                        feature  importance
0                            life_form_geophyte    0.173751
1             Stace_Crawley_nativity_aliens_Neo    0.152295
2  life_form_epiphyte / parasite / phanerophyte    0.091432
3                                     seed_mass    0.050655
4                                           SLA    0.029854
5                              growth_form_Fern    0.029302
6                               mean_veg_height    0.026023
7                                    L_PLANTATT    0.024242
8                                          LDMC    0.021859
9                                     leaf_area    0.021213
Optimal number of features: 373
Selected features: ['SLA', 'LDMC', 'seed_mass', 'leaf_area', 'mean_veg_height', 'max_veg_height', 'L_PLANTATT', 'F_PLANTATT', 'R_PLANTATT', 'N_PLANTATT', 'S_PLANTATT', 'L_Doring', 'F_Doring', 'R_Doring', 'N_Doring', 'S_Doring', 'T_Doring', 'GB_Man_

In [61]:
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.pipeline import Pipeline

def run_models(X, y, X_dict, n_estimators=100, cv=5):
    results = {}
    
    if not isinstance(y, pd.DataFrame):
        y = pd.DataFrame(y, columns=['target'])
    
    for target in y.columns:
        print(f"\nModeling for target: {target}")
        y_target = y[target].dropna()
        X_target = X_dict[target]
        X_all = X.loc[y_target.index]
        
        models = {
            'Random Forest (Selected Features)': RandomForestRegressor(n_estimators=n_estimators, random_state=42),
            'Gradient Boosting (Selected Features)': HistGradientBoostingRegressor(max_iter=n_estimators, random_state=42),
            'Random Forest (All Features)': RandomForestRegressor(n_estimators=n_estimators, random_state=42),
            'Gradient Boosting (All Features)': HistGradientBoostingRegressor(max_iter=n_estimators, random_state=42)
        }
        
        target_results = {}
        
        for name, model in models.items():
            if 'Selected Features' in name:
                X_use = X_target
            else:
                X_use = X_all
            
            # Cross-validation
            scores = cross_val_score(model, X_use, y_target, cv=cv, scoring='neg_mean_squared_error')
            rmse_scores = np.sqrt(-scores)
            
            # Train on full dataset and get R2 score
            model.fit(X_use, y_target)
            y_pred = model.predict(X_use)
            r2 = r2_score(y_target, y_pred)
            
            target_results[name] = {
                'RMSE': rmse_scores.mean(),
                'RMSE_std': rmse_scores.std() * 2,
                'R2': r2
            }
            
            print(f"{name}:")
            print(f"  RMSE: {rmse_scores.mean():.4f} (+/- {rmse_scores.std() * 2:.4f})")
            print(f"  R2 Score: {r2:.4f}")
        
        results[target] = target_results
    
    return results

# Run the models
model_results = run_models(X_selected_drop, y, X_selected_dict_drop)

# Print summary
print("\nSummary of Results:")
for target, target_results in model_results.items():
    print(f"\nTarget: {target}")
    for model, metrics in target_results.items():
        print(f"  {model}:")
        print(f"    RMSE: {metrics['RMSE']:.4f} (+/- {metrics['RMSE_std']:.4f})")
        print(f"    R2 Score: {metrics['R2']:.4f}")


Modeling for target: GS_1C_pg
Random Forest (Selected Features):
  RMSE: 4.9008 (+/- 1.0418)
  R2 Score: 0.9131
Gradient Boosting (Selected Features):
  RMSE: 4.9584 (+/- 0.9897)
  R2 Score: 0.8400
Random Forest (All Features):
  RMSE: 4.8928 (+/- 1.0009)
  R2 Score: 0.9124
Gradient Boosting (All Features):
  RMSE: 4.9448 (+/- 1.0045)
  R2 Score: 0.8423

Modeling for target: GS_2C_pg
Random Forest (Selected Features):
  RMSE: 9.7858 (+/- 2.0380)
  R2 Score: 0.9126
Gradient Boosting (Selected Features):
  RMSE: 9.8975 (+/- 2.0371)
  R2 Score: 0.8435
Random Forest (All Features):
  RMSE: 9.8471 (+/- 2.0763)
  R2 Score: 0.9123
Gradient Boosting (All Features):
  RMSE: 9.9387 (+/- 2.0348)
  R2 Score: 0.8431

Modeling for target: GS_1C_Mbp
Random Forest (Selected Features):
  RMSE: 4804.9114 (+/- 1016.1759)
  R2 Score: 0.9104
Gradient Boosting (Selected Features):
  RMSE: 4823.3083 (+/- 1014.9051)
  R2 Score: 0.8405
Random Forest (All Features):
  RMSE: 4796.1128 (+/- 1036.2625)
  R2 Score

KeyboardInterrupt: 